In [1]:
%pip install numpy==1.23.5 pandas==1.5.3 scipy==1.10.1 TensorFlow==2.11.0

  Using cached numpy-1.23.5-cp39-cp39-win_amd64.whl (14.7 MB)
  Using cached pandas-1.5.3-cp39-cp39-win_amd64.whl (10.9 MB)
     -------------------------------------- 266.3/266.3 MB 3.7 MB/s eta 0:00:00
     ------------------------------------- 439.2/439.2 kB 28.6 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 54.0 MB/s eta 0:00:00
     ---------------------------------------- 895.9/895.9 kB ? eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 29.5 MB/s eta 0:00:00
     ------------------------------------- 781.3/781.3 kB 24.9 MB/s eta 0:00:00
     ---------------------------------------- 216.1/216.1 kB ? eta 0:00:00
     ---------------------------------------- 160.1/160.1 kB ? eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: protobuf


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\gopir\\anaconda3\\Lib\\site-packages\\google\\~upb\\_message.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [9]:
!pip install opencv-python==4.8.1.78

     --------------------------------------- 38.1/38.1 MB 15.6 MB/s eta 0:00:00


In [2]:
!pip install torch==2.2.2

     -------------------------------------- 198.5/198.5 MB 3.8 MB/s eta 0:00:00


In [4]:
!pip install torchvision==0.17.2

     ---------------------------------------- 1.2/1.2 MB 9.2 MB/s eta 0:00:00


In [62]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Data path and transformations
path = r"C:\Users\gopir\Desktop\Plant_disease_detection\Data\NewPlant DiseasesDataset\NewPlantDiseases\train"
transformation = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data = datasets.ImageFolder(root=path, transform=transformation)

# Split data
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = random_split(data, [train_size, test_size])

# Data loaders
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=32, num_workers=4, pin_memory=True)

# CNN model
class CNN_Classification(nn.Module):
    def __init__(self):
        super(CNN_Classification, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, len(data.classes))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initiate model, loss, optimizer
CNN_Model = CNN_Classification().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CNN_Model.parameters(), lr=0.001)

# Accuracy evaluation function
def evaluate_metrics(model, dataloader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    model.train()
    report = classification_report(all_labels, all_preds, target_names=class_names, digits=4, zero_division=0)
    return report



# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    CNN_Model.train()

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = CNN_Model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"Avg Loss: {avg_loss:.4f}")
    
    print("Evaluation on Test Set:")
    metrics_report = evaluate_metrics(CNN_Model, test_loader, device, data.classes)
    print(metrics_report)

Using device: cpu

Epoch 1/10
Avg Loss: 1.0286
Evaluation on Test Set:
                                                    precision    recall  f1-score   support

                                Apple___Apple_scab     0.8741    0.8138    0.8429       435
                                 Apple___Black_rot     0.8853    0.8157    0.8491       407
                          Apple___Cedar_apple_rust     0.9446    0.7521    0.8374       363
                                   Apple___healthy     0.7911    0.7717    0.7813       368
                               Blueberry___healthy     0.9149    0.8202    0.8649       367
          Cherry_(including_sour)___Powdery_mildew     0.9250    0.8783    0.9011       337
                 Cherry_(including_sour)___healthy     0.9762    0.6649    0.7910       370
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot     0.7606    0.8491    0.8024       318
                       Corn_(maize)___Common_rust_     0.9874    0.9654    0.9763       405
        

In [63]:
# Load a test image and preprocess it
img = Image.open(r"C:\Users\gopir\Desktop\Plant_disease_detection\Data\test\test\AppleCedarRust4.JPG")
img = transform(img).unsqueeze(0)  # add batch dimension

# Pass the image through the model
CNN_Model.eval()
output = CNN_Model(img)
_, predicted = torch.max(output, 1)
print(f'Predicted class: {data.classes[predicted.item()]}')

Predicted class: Apple___Cedar_apple_rust


In [64]:
# Load a test image and preprocess it
img = Image.open(r"C:\Users\gopir\Desktop\Plant_disease_detection\Data\test\test\TomatoYellowCurlVirus4.JPG")
img = transform(img).unsqueeze(0)  # add batch dimension

# Pass the image through the model
CNN_Model.eval()
output = CNN_Model(img)
_, predicted = torch.max(output, 1)
print(f'Predicted class: {data.classes[predicted.item()]}')

Predicted class: Tomato___Tomato_Yellow_Leaf_Curl_Virus


In [65]:
torch.save(CNN_Model.state_dict(), 'cnn_model.pth')